In [1]:
from pathlib import Path
import zipfile
import pandas as pd
import urllib.request
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import seaborn as sb
import plotly.express as px
sklearn.set_config(display="diagram")

np.random.seed(42)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)



import pandas as pd

def load_data():
  return pd.read_csv(Path("dataset.csv"))

#Load the data set
data = load_data()


In [2]:
data.head()

ID  travel_fee             departure_time  departure_long  departure_lat  arrival_long  arrival_lat  occupancy
0  2013-07-02 19:54:00.000000232         7.0  2013-07-02 19:54:00+00:00      -74.005360      40.728867    -74.008913    40.710907          1
1    2013-09-28 00:21:31.0000002         5.5  2013-09-28 00:21:31+00:00      -74.014165      40.708941    -74.016310    40.716734          1
2  2013-06-16 03:18:00.000000150        21.5  2013-06-16 03:18:00+00:00      -73.991075      40.760352    -73.941382    40.713292          1
3  2013-07-20 13:43:00.000000121         9.5  2013-07-20 13:43:00+00:00      -74.002662      40.723630    -73.991722    40.748905          5
4    2013-11-05 22:57:17.0000003        15.5  2013-11-05 22:57:17+00:00      -73.962397      40.712705    -73.996834    40.680403          2

In [3]:
data.describe()

travel_fee  departure_long  departure_lat  arrival_long   arrival_lat     occupancy
count  2.596558e+06    2.596558e+06   2.596558e+06  2.596514e+06  2.596514e+06  2.596558e+06
mean   1.259551e+01   -7.243741e+01   3.974318e+01 -7.239685e+01  3.972336e+01  1.709320e+00
std    1.081654e+01    1.125115e+01   9.133454e+00  1.143503e+01  8.438912e+00  1.375425e+00
min   -5.200000e+01   -7.400217e+02  -3.124490e+03 -1.216417e+03 -3.111707e+03  0.000000e+00
25%    6.500000e+00   -7.399220e+01   4.073442e+01 -7.399147e+01  4.073352e+01  1.000000e+00
50%    9.500000e+00   -7.398186e+01   4.075236e+01 -7.398023e+01  4.075279e+01  1.000000e+00
75%    1.400000e+01   -7.396680e+01   4.076704e+01 -7.396362e+01  4.076779e+01  2.000000e+00
max    5.000000e+02    8.343336e+01   2.342817e+03  1.428740e+03  4.739714e+02  9.000000e+00

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2596558 entries, 0 to 2596557
Data columns (total 8 columns):
 #   Column          Dtype  
---  ------          -----  
 0   ID              object 
 1   travel_fee      float64
 2   departure_time  object 
 3   departure_long  float64
 4   departure_lat   float64
 5   arrival_long    float64
 6   arrival_lat     float64
 7   occupancy       int64  
dtypes: float64(5), int64(1), object(2)
memory usage: 158.5+ MB


In [5]:
from sklearn.model_selection import train_test_split


train, test = train_test_split(data, test_size=0.20, random_state=42)

print("Length of train_set:", len(train))
print("Length of test_set:", len(test))

Length of train_set: 2077246
Length of test_set: 519312


In [7]:
#from sys import orig_argv
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder


class HaversineDistanceTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        distances = haversine_distance(X['departure_lat'], X['departure_long'],
                                       X['arrival_lat'], X['arrival_long'])
        return np.c_[X, distances]

num_attrs = ['departure_long','departure_lat','arrival_long','arrival_lat','occupancy']
cat_attrs = ['departure_time']

num_pipeline = make_pipeline(
    SimpleImputer(strategy='mean'),
    #FunctionTransformer(np.log1p, validate=False),
    StandardScaler()
)

cat_pipeline = make_pipeline(
 SimpleImputer(strategy="most_frequent"),
 OneHotEncoder(handle_unknown="ignore")
)


distance_pipeline = make_pipeline(
    HaversineDistanceTransformer()
)

preprocessing = ColumnTransformer([
    ('num', num_pipeline, num_attrs),
    ('cat', cat_pipeline, cat_attrs),
    ('distance', distance_pipeline,[])
])

In [9]:
taxi = train.drop('travel_fee',axis=1)
taxi_fee = train['travel_fee'].copy()

In [10]:
taxi_prepared = preprocessing.fit_transform(taxi)

In [11]:
taxi_prepared

<2077246x1210239 sparse matrix of type '<class 'numpy.float64'>'
	with 12463476 stored elements in Compressed Sparse Row format>

In [12]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.compose import TransformedTargetRegressor
from sklearn.pipeline import Pipeline


# Splitting the data into features (X) and target variable (y)
X = taxi
y = taxi_fee

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Creating a Ridge Regression model
ridge_model = Ridge(alpha=1.0)


# Creating a pipeline that combines preprocessing and Ridge Regression
full_pipeline = Pipeline([
    ('preprocessing', preprocessing),  # Your existing preprocessing steps
    ('ridge', TransformedTargetRegressor(regressor=ridge_model,
                                         transformer=StandardScaler())),
])

# Training the combined pipeline
full_pipeline.fit(X_train, y_train)


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer()),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  ['departure_long',
                                                   'departure_lat',
                                                   'arrival_long',
                                                   'arrival_lat',
                                                   'occupancy']),
                                                 ('cat',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['departure_time']),
                                                 ('distance',
                                                  Pipeline(steps=[('haversinedistancetransformer',
                                                                   HaversineDistanceTransformer())]),
                                                  [])])),
                ('ridge',
                 TransformedTargetRegressor(regressor=Ridge(),
                                            transformer=StandardScaler()))])

In [13]:
# Making predictions
y_train_pred = full_pipeline.predict(X_train)
y_test_pred = full_pipeline.predict(X_test)

# Evaluating the model
train_rmse = mean_squared_error(y_train, y_train_pred, squared=False)
test_rmse = mean_squared_error(y_test, y_test_pred, squared=False)

print(f"Train RMSE: {train_rmse:.2f}")
print(f"Test RMSE: {test_rmse:.2f}")

Train RMSE: 7.77
Test RMSE: 11.28


In [14]:
from sklearn.metrics import mean_absolute_error

# Making predictions using the trained pipeline
y_train_pred = full_pipeline.predict(X_train)
y_test_pred = full_pipeline.predict(X_test)

# Calculating the Mean Absolute Error (MAE)
train_mae = mean_absolute_error(y_train, y_train_pred)
test_mae = mean_absolute_error(y_test, y_test_pred)

print("Train MAE:", train_mae)
print("Test MAE:", test_mae)


Train MAE: 4.728327872418664
Test MAE: 7.108336533307537


In [15]:
from sklearn.model_selection import GridSearchCV

# Create a parameter grid to search for the best alpha value
param_grid = {'ridge__regressor__alpha': [0.001, 0.01, 0.1, 1, 10, 100]}

# Create a GridSearchCV object
grid_search = GridSearchCV(full_pipeline, param_grid, cv=5, scoring='neg_mean_squared_error')

# Fit the grid search on the training data
grid_search.fit(X_train, y_train)

# Get the best parameter
best_alpha = grid_search.best_params_['ridge__regressor__alpha']

# Print the best parameter
print(f"Best alpha: {best_alpha}")

# Get the best estimator from the grid search
best_estimator = grid_search.best_estimator_

# Make predictions using the best estimator
y_train_pred = best_estimator.predict(X_train)
y_test_pred = best_estimator.predict(X_test)

# Calculate and print the RMSE and MAE
train_rmse = mean_squared_error(y_train, y_train_pred, squared=False)
test_rmse = mean_squared_error(y_test, y_test_pred, squared=False)

train_mae = mean_absolute_error(y_train, y_train_pred)
test_mae = mean_absolute_error(y_test, y_test_pred)

print(f"Train RMSE: {train_rmse:.2f}")
print(f"Test RMSE: {test_rmse:.2f}")
print("Train MAE:", train_mae)
print("Test MAE:", test_mae)


Best alpha: 100
Train RMSE: 10.72
Test RMSE: 10.82
Train MAE: 6.722101247636019
Test MAE: 6.794638946164586
